In [23]:
import pandas as pd
import numpy as np
from __future__ import division
from scipy.spatial.distance import euclidean
from sklearn.metrics import roc_auc_score

data = pd.read_csv('other_files/data-logistic.csv', names=['Class', '1', '2'])
X = data.drop('Class', axis=1)
y = data['Class']
L = len(X)
#X = data.drop('Class', axis=1)[:180]
#y = data['Class'][:180]
#L = len(X)
#X_train = data.drop('Class', axis=1)[-25:]
#y_train = data['Class'][-25:]
#L_train = len(X_train)
#X_all = data.drop('Class', axis=1)
#y_all = data['Class']
#L_all = len(X_all)
C = 10
k = 0.1
w_interval = 0.00001

In [14]:
def regression_eq(X, y, w1, w2):
    Sum = 0;
    for i in xrange(0, L):
        Sum += np.log(1+np.exp(-1*y[0]*(w1*X['1'][i] + w2*X['2'][i])))
    return (1/L)*Sum

def regul_regression_eq(X, y, w1, w2):
    Sum = 0;
    for i in xrange(0, L):
        Sum += np.log(1+np.exp(-1*y[0]*(w1*X['1'][i] + w2*X['2'][i])))
    return (1/L)*Sum + (1/2)*C*(np.linalg.norm([w1,w2]))

def w1_diff(X, y, w1, w2):
    Sum = 0;
    for i in xrange(0, L):
        Sum += y[i]*X['1'][i]*(1-(1/(1+np.exp(-1*y[i]*(w1*X['1'][i] + w2*X['2'][i])))))
    return w1 + k*(1/L)*Sum

def regul_w1_diff(X, y, w1, w2):
    Sum = 0;
    for i in xrange(0, L):
        Sum += y[i]*X['1'][i]*(1-(1/(1+np.exp(-1*y[i]*(w1*X['1'][i] + w2*X['2'][i])))))
    return w1 + k*(1/L)*Sum - k*C*w1

def w2_diff(X, y, w1, w2):
    Sum = 0;
    for i in xrange(0, L):
        Sum += y[i]*X['2'][i]*(1-(1/(1+np.exp(-1*y[i]*(w1*X['1'][i] + w2*X['2'][i])))))
    return w2 + k*(1/L)*Sum

def regul_w2_diff(X, y, w1, w2):
    Sum = 0;
    for i in xrange(0, L):
        Sum += y[i]*X['2'][i]*(1-(1/(1+np.exp(-1*y[i]*(w1*X['1'][i] + w2*X['2'][i])))))
    return w2 + k*(1/L)*Sum - k*C*w2

In [27]:
w1 = 0
w2 = 0
for i in xrange(0, 10000):
    w1_new = w1_diff(X, y, w1, w2)
    w2_new = w2_diff(X, y, w1, w2)
    if (euclidean([w1, w2], [w1_new, w2_new]) >= w_interval):
        w1 = w1_new
        w2 = w2_new
    else:
        break
        
y_score = []
for i in xrange(0, L):
    y_score.append(1/(1 + np.exp(-1*X['1'][i]*w1 - X['2'][i]*w2)))

w1 = 0
w2 = 0
for i in xrange(0, 10000):
    w1_new = regul_w1_diff(X, y, w1, w2)
    w2_new = regul_w2_diff(X, y, w1, w2)
    if (euclidean([w1, w2], [w1_new, w2_new]) >= w_interval):
        w1 = w1_new
        w2 = w2_new
    else:
        break
        
regul_y_score = []
for i in xrange(0, L):
    regul_y_score.append((1/(1 + np.exp(-1*X['1'][i]*w1-X['2'][i]*w2))))

In [26]:
print np.round(roc_auc_score(y, y_score), 3)
print np.round(roc_auc_score(y, regul_y_score), 3)

0.927
0.936
